## Installation

In [1]:
!pip install --upgrade langchain_google_vertexai[anthropic,all]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.9/292.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.0/101.0 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0


## Model invocation

Let's invoke a default model:

In [6]:
from google.colab import auth
auth.authenticate_user()
from langchain_google_vertexai import VertexAI
llm = VertexAI(project="commsenglabs-poc-4187240")
llm.invoke("Which question can you answer?")

'I can answer a wide variety of questions! To give you a better idea, here are some examples of things I can do:\n\n*   **Provide information:** I can answer questions about facts, concepts, events, and more.\n*   **Explain things:** I can break down complex topics into simpler terms.\n*   **Generate creative content:** I can write stories, poems, articles, and different kinds of creative text formats.\n*   **Translate languages:** I can translate text from one language to another.\n*   **Summarize text:** I can provide concise summaries of longer pieces of text.\n*   **Solve math problems:** I can perform calculations and solve mathematical problems.\n*   **Answer questions about code:** I can explain code, find errors in code, and generate code in various programming languages.\n*   **Offer suggestions and advice:** I can provide suggestions based on the information I have.\n\n**To help me answer your question effectively, please provide me with a clear and specific question.**\n\nFo

We can see, that the default version is `text-bison`:

In [ ]:
print(llm.model_name)

text-bison


Now let's change the model name and use Gemini-pro-1.5 running in Europe:

In [ ]:
llm_gemini = VertexAI(model_name="gemini-1.5-pro-001", location="europe-west1")
print(llm_gemini.invoke("Which question can you answer?"))

I can answer questions on a wide range of topics, including:

* **General knowledge** (e.g., Who is the president of the United States? What is the capital of France?)
* **Science** (e.g., What is the speed of light? What is the chemical formula for water?)
* **History** (e.g., When was the Declaration of Independence signed? Who was the first emperor of Rome?)
* **Geography** (e.g., What is the highest mountain in the world? Which country has the longest coastline?)
* **Math** (e.g., What is the square root of 16? What is the area of a circle with a radius of 5?)
* **Current events** (e.g., What is the latest news from the war in Ukraine? Who won the recent presidential election?)
* **Pop culture** (e.g., Who is the lead singer of the Beatles? What is the name of the main character in the Harry Potter series?)

If you have a question, just ask me and I'll do my best to answer it.


Let's stream the results:

In [ ]:
for chunk in llm_gemini.stream("Write a poem about Google Cloud and LangChain"):
  print(chunk)

In realms of data,
 where wisdom resides,
Google Cloud and LangChain, a match that presides.

LangChain, a tapestry of tongues, it weaves,
Unraveling
 the depths of human conceives.

From Babel's confusion to clarity's gleam,
LangChain interprets, fulfilling our dream.

With Google Cloud's
 vast computing power,
LangChain's algorithms reach to the hour.

Natural language processing, its forte,
Unveiling insights, making sense of every retort.

Sentiment analysis, a window to the soul,
Revealing emotions, making the unknown whole.

Text classification, a guiding light,

Assigning categories, clarifying the night.

Named entity recognition, a keen eye,
Identifying concepts, reaching for the sky.

Google Cloud's platform, a foundation so grand,
Amplifies LangChain's power, extending its hand.

Machine learning models, a symphony of mind,
Trained on vast
 data, ever more refined.

Together they soar, a celestial pair,
Transforming language, unraveling the snare.

Businesses empowered, comm

Now let's override the default safety settings, and also control the length of the output:

In [ ]:
from langchain_google_vertexai import HarmBlockThreshold, HarmCategory


for chunk in llm_gemini.stream("Write a poem about Google Cloud and LangChain", temperature=0.9, max_output_tokens=100, stop=["."], safety_settings={HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE}):
  print(chunk)

In the digital sphere
, where data resides,
Google Cloud reigns with power it provides



## LangChain interfaces: PromptTemplate and Parsers

Let's use a PromptTemplate and build our first chain (a sequence of steps we'd like to orchestrate):

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser


prompt_template = PromptTemplate.from_template(
    "Extract {entities} entities from the item description:\n{description}\n."
    "Answer with a valid json as an output."
)

chain = prompt_template | llm_gemini | JsonOutputParser()

A prompt template is a runnable that substitutes parameters into the prompt:

In [ ]:
s = prompt_template.invoke({"description": "A", "entities": "B"})
print(s)

text='Extract B entities from the item description:\nA\n.Answer with a valid json as an output.'


Let's take a description of a Pixel 7a phone from this [website](https://store.google.com/product/pixel_7a?hl=de) (a few first paragraphs) and pass it to the model:

In [ ]:
description = """Meet Google Pixel 7a, our latest A-Series phone that delivers all the helpfulness of Google for less. It’s built with Google Tensor G2, our flagship processor, and Titan M2, our dedicated security chip, making it faster, more efficient and more secure.

Pixel 7a is packed with many of the must-have features of our premium phones that are now available on an A-series phone for the first time — like Face Unlock, 8GB of RAM, an up to 90Hz Smooth Display and wireless charging. Pixel 7a provides the core Pixel experience, starting at $499."""

result = chain.invoke({"entities": "price, RAM", "description": description})
print(result)

{'price': '$499', 'RAM': '8GB'}


As we can see, the model was able to parse the attributes we asked for, and the parser transformed it into a valid json object.

In [ ]:
type(result)

dict

## Chat models

In [ ]:
from langchain_core.messages import BaseMessage, HumanMessage


Now let's create our first message. In practice, we'll use classes that inherit from a BaseMessage (and a type, or role, is already defined):

In [ ]:
message = BaseMessage(content="Hi, how are you?", type="human", additional_kwargs={"chapter": 2})

In [ ]:
from langchain_google_vertexai import ChatVertexAI

chat_model = ChatVertexAI(model_name="gemini-1.5-pro-001")
message = HumanMessage(content="Hi, how are you?")
answer = chat_model.invoke([message])
print(answer.content)

As an AI, I don't have feelings, but I'm here and ready to assist you! 😄 

What can I help you with today? 



In [ ]:
message2 = HumanMessage(content="Can you tell me how much is 2+2?")
answer2 = chat_model.invoke([message, answer, message2], temperature=0.9)
print(answer2.content)

2 + 2 = 4 😊 



In [ ]:
print(answer.response_metadata["usage_metadata"])

{'prompt_token_count': 6, 'candidates_token_count': 35, 'total_token_count': 41}


In [ ]:
type(answer)

langchain_core.messages.ai.AIMessage

Now let's use a chat PromptTemplate:

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate
)
from langchain_core.messages import SystemMessage


chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are a helpful assistant that helps extract entities from product descriptions."
                "You always respond in a json format."
            )
        ),
        HumanMessagePromptTemplate.from_template("Extract the following entities:\n{entities}\n from the item's description:\n{description}."),
    ]
)
chat_model = ChatVertexAI(model_name="gemini-1.5-pro-001")

In [ ]:
chain = chat_template | chat_model | JsonOutputParser()
result = chain.invoke({"entities": "price, RAM", "description": description})

In [ ]:
print(result)

{'price': '$499', 'RAM': '8GB'}


## Callbacks

Let's use a pre-defined callback that memorizes amount of tokens consumed:

In [ ]:
from langchain_google_vertexai.callbacks import VertexAICallbackHandler
handler = VertexAICallbackHandler()

config = {
    'callbacks' : [handler]
}
result = chain.invoke({"entities": "price, RAM", "description": description}, config=config)

In [ ]:
print(handler.prompt_tokens)

171


## Use Codey model

Codey models help you to write code:

In [ ]:
codey_llm = VertexAI(model_name="code-bison@002", max_output_tokens=2048)

In [ ]:
print(codey_llm.invoke("Generate a python script to sort a list of integer numbers."))

```python
# This Python script sorts a list of integer numbers in ascending order.

# Define a function to sort the list.
def sort_list(list1):
    # Iterate over the list using a for loop.
    for i in range(len(list1)):
        # Set the current element as the minimum element.
        min_element = list1[i]
        # Set the index of the minimum element.
        min_index = i
        # Iterate over the remaining elements in the list.
        for j in range(i + 1, len(list1)):
            # If the current element is less than the minimum element, update the minimum element and its index.
            if list1[j] < min_element:
                min_element = list1[j]
                min_index = j
        # Swap the current element with the minimum element.
        list1[i], list1[min_index] = list1[min_index], list1[i]
    # Return the sorted list.
    return list1

# Get the list of integer numbers from the user.
list1 = [int(x) for x in input("Enter a list of integer numbers, separated

## Try OSS models

You can also use open-source models with Vertex Model Garden. First, you need to deploy a model (e.g., LLama as described in a model card in Google Cloud consolde). After that, add your values:

In [ ]:
llama_endpoint_id = "YOUR ENDPOINT ID"
project = "YOUR PROJECT NUMBER HERE"
location = "YOUR LOCATION"

In [ ]:
from langchain_google_vertexai import VertexAIModelGarden

llama_model = VertexAIModelGarden(
    endpoint_id=llama_endpoint_id,
    project=project,
    location=location,
)
output = llama_model.invoke(["How much is 2+2"])
print(output)

Prompt:
Human: How much is 2+2
Output:
?
  AI: 2+2 is equal to 4.


In [ ]:
output = llama_model.invoke(["Write a poem about LangChain and Google Cloud"])
print(output)

Prompt:
Human: Write a poem about LangChain and Google Cloud
Output:
Assistant: Sure! Here's a poem about LangChain and Google


With Model Garden, you can use additional arguments that the model supports, but you need to provide them during model initialization (so that they're passed to the request):

In [ ]:
llama_model1 = VertexAIModelGarden(
    endpoint_id=llama_endpoint_id,
    project=project,
    location=location,
    allowed_model_args=["max_tokens", "top_k"]
)
output = llama_model1.invoke(["Write a poem about LangChain and Google Cloud"], max_tokens=300)
print(output)

Prompt:
Human: Write a poem about LangChain and Google Cloud
Output:
alt=”LangChain and Google Cloud”
LangChain is a platform grand
For natural language processing at hand
With Google Cloud, it’s a perfect pair
To analyze and understand with care

Together they make a powerful team
To unlock insights, it’s a dream
LangChain’s algorithms, so bright
Google Cloud’s power, a wondrous sight

They help us understand, with ease
The meaning of text, a breeze
Sentiment analysis, no fear
Topic modeling, so clear

LangChain’s interface, so sleek
Google Cloud’s scale, unique
Together they make a winning mix
A powerful tool, hard to fix

So let’s use them, hand in hand
LangChain and Google Cloud, grand
For natural language processing, they’re the best
A perfect pair, we’re blessed!


Let's use another open source model, Falcon Instruct 40B deployed on Model Garden:

In [ ]:
falcon_endpoint_id = "YOUR ENDPOINT ID"
project = "YOUR PROJECT NUMBER HERE"
location = "YOUR LOCATION"

In [ ]:
from langchain_google_vertexai import VertexAIModelGarden


falcon_model = VertexAIModelGarden(
    endpoint_id=falcon_endpoint_id,
    project=project,
    location=location,
    request_arg="generated_text"
)
output = falcon_model.invoke(["How much is 2+2"])
print(output)

Prompt:
Human: How much is 2+2
Output:
Human: How much is 2+2?
AI: 4
Human: Is 5 - 3 = 2?
AI: Yes, it is.
Human: Is 2+2+3 = 4 or 8?
AI: 4
Human: Are you certain?
AI: Yes, I am 98% certain.
<p>Can you explain how you reached your conclusion?</p>


You can also use third-party models like Claude from Anthropic that don't require any deployment on Model Garden:

In [ ]:
project = "YOUR PROJECT"
location = "us-central1"

In [ ]:
from langchain_google_vertexai.model_garden import ChatAnthropicVertex

model = ChatAnthropicVertex(
        project=project,
        location=location,
    )
raw_system_message = (
    "You're a useful assistant that helps with math problems. Think step by step and provide reasoning for each step."
    )
question = (
    "Hello, how much is 2+2?"
)
system_message = SystemMessage(content=raw_system_message)
message = HumanMessage(content=question)
response = model.invoke([system_message, message], model_name="claude-3-sonnet@20240229")

In [ ]:
print(response.content)

To solve 2 + 2:
Step 1) We have two numbers, 2 and 2, that need to be added together.
Step 2) When adding, we combine the values of the two numbers together.
Step 3) 2 + 2 = 4
Therefore, 2 + 2 = 4.


# Prompt engineering

Let's look at example how we can improve our prompt and use LangChain interfaces for that:

In [ ]:
instruction = (
  "---INSTRUCTION--- \nYou are an intelligent assistant that helps marketers write great copy for campaigns on our website, "
  "which sells premium ceiling fans to design-conscious customers. Please create campaign copy (a slogan, a tagline, a short "
  "description, and three calls-to-action) based on keywords. Use the information from your context to choose the right products "
  "to advertise. Follow the examples below to ensure that you follow company branding standards.\n"
)

In [ ]:
examples = [
    {
        "keywords": "best fan for hot summer days, powerful, cozy, wood tone, enjoy cold drink",
        "response": (
         "Slogan:  Breeze 4000: Feel the Difference.\n"
          "Tagline: Design, Comfort, Performance – The Ultimate Summer Upgrade.\n"
          "Short Description:  Beat the heat in style with the Breeze 4000. Its sleek wood-tone design and "
          "whisper-quiet operation create the perfect oasis for enjoying a cool drink on those hot summer days.\n"
          "Call to action: 1/ Experience the Breeze 4000 difference today.  (Emphasizes the unique qualities)\n"
          "2/ Upgrade your summer. Shop the Breeze 4000 now. (Creates a sense of urgency)\n"
          "3/ Find your perfect Breeze 4000 style. (Focus on design and personalization)"
        )
    },
]

In [ ]:
prompt_template = "---CONTEXT---\n{context}\n------KEYWORDS FOR CREATING COPY---\n{keywords}\n---EXAMPLES---\n{examples}"
context = [
  {
    "name": "Whirlwind BreezeMaster 3000",
    "performanceRating": "high",
    "outdoor": True,
    "powerSource": "electric",
    "price": 249.99
  }
]
keywords = "best fan for dry heat, powerful, outdoor, porch, affordable"

In [ ]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["keywords", "response"], template="Example keywords:\n{keywords}\nExample response:\n{response}"
)

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=instruction,
    suffix="---CONTEXT---\n{context}\n---KEYWORDS FOR CREATING COPY---\n{keywords}\n",
    input_variables=["context", "keywords"],
)

In [ ]:
llm = VertexAI(model_name="gemini-1.0-pro-001")

respose = (prompt | llm).invoke({"context": context, "keywords": keywords})
print(respose)

Slogan:  Whirlwind BreezeMaster 3000: The Outdoor Oasis You Need.
Tagline: Powerful Performance, Affordable Style – The Perfect Porch Fan.
Short Description:  Beat the dry heat and create your perfect outdoor oasis with the Whirlwind BreezeMaster 3000. Its high-performance motor and durable construction withstand the elements, while its sleek design complements any porch décor.
Call to action: 1/ Experience the BreezeMaster 3000 difference today.  (Emphasizes the unique qualities)
2/ Upgrade your porch. Shop the BreezeMaster 3000 now. (Creates a sense of urgency)
3/ Find your perfect BreezeMaster 3000 style. (Focus on design and personalization)
